In [60]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotZoo
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random
using Plots
include("quadruped.jl")
include("quadratic_cost.jl")
include("moi.jl")
include("sparseblocks.jl")

  Activating environment at `~/Desktop/16745_OCRL/Project/Optimal-Control-Project/Project.toml`
Precompiling project...
  ✓ Wayland_protocols_jll
  ✓ JSONSchema
  ✓ OpenSSL_jll
  ✓ Expat_jll
  ✓ LERC_jll
  ✓ ChainRulesCore
  ✓ ArrayInterface
  ✓ Wayland_jll
  ✓ Fontconfig_jll
  ✓ Libtiff_jll
  ✓ StructArrays
  ✓ ChangesOfVariables
  ✓ xkbcommon_jll
  ✓ WebIO
  ✓ Cairo_jll
  ✓ MUMPS_seq_jll
  ✓ Qt5Base_jll
  ✓ HarfBuzz_jll
  ✓ FiniteDiff
  ✓ RecipesPipeline
  ✓ LogExpFunctions
  ✓ libass_jll
  ✓ Interpolations
  ✓ CSSUtil
  ✓ Ipopt_jll
  ✓ FFMPEG_jll
  ✓ JSExpr
  ✓ FFMPEG
  ✓ PlotThemes
  ✓ StatsBase
  ✓ GR_jll
  ✓ Blink
  ✓ SpecialFunctions
  ✓ Knockout
  ✓ GeometryBasics
  ✓ DiffRules
  ✓ GR
  ✓ MeshIO
  ✓ InteractBase
  ✓ ForwardDiff
  ✓ MeshCat
  ✓ SparseDiffTools
  ✓ MechanismGeometries
  ✓ RobotDynamics
  ✓ MeshCatMechanisms
  ✓ RobotZoo
  ✓ MathOptInterface
  ✓ Plots
  ✓ Ipopt
  ✓ OSQP
  ✓ TrajectoryOptimization
  ✓ Altro
  ✓ TrajOptPlots
  53 dependencies successfully precompile

solve

## Dynamics Models

In [193]:
include("simple_quadruped.jl")
full_model = UnitreeA1()
n,m = state_dim(full_model), control_dim(full_model)
mvis = initialize_visualizer(full_model);
model = simplifyQuadruped(full_model)
n,m = state_dim(model), control_dim(model);

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8709
└ @ MeshCat C:\Users\pnadan\.julia\packages\MeshCat\Ax8pH\src\visualizer.jl:73


## Reference Trajectory & Contact Sequence

In [76]:
include("ref_trajectory.jl")
include("gait.jl")

# Discretization
tf = 4.4
dt = 0.1
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N);

# Reference Trajectory
Xref,Uref = reference_trajectory(model, times)

# Contact Sequence
contact_sched = contact_sequence_trot(times)

4×45 Matrix{Float64}:
 1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0  …  0.0  1.0  1.0  1.0  1.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0     1.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0     1.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  1.0  1.0  1.0  0.0  0.0  0.0  0.0     0.0  1.0  1.0  1.0  1.0  1.0  0.0

## Define NLP for IPOPT

In [84]:
include("nlp.jl")
include("nlp_cost.jl")
include("nlp_constraint.jl")

## Solve NLP

## Controller

## Simulation

   Resolving package versions...
   Installed TimerOutputs ──── v0.5.17
   Installed SparseDiffTools ─ v1.21.0
    Updating `C:\Users\pnadan\Documents\GitHub\Optimal-Control-Project\Project.toml`
  [6ad125db] + MeshCatMechanisms v0.8.2
    Updating `C:\Users\pnadan\Documents\GitHub\Optimal-Control-Project\Manifest.toml`
  [4fba245c] ↑ ArrayInterface v4.0.4 ⇒ v5.0.7
  [70588ee8] + CSSUtil v0.1.1
  [3da002f7] ↓ ColorTypes v0.11.0 ⇒ v0.10.12
  [34da2185] ↑ Compat v3.42.0 ⇒ v3.43.0
  [d3863d7c] + InteractBase v0.10.8
  [a98d9a8b] ↑ Interpolations v0.13.5 ⇒ v0.13.6
  [bcebb21b] + Knockout v0.2.5
  [2ab3a3ac] ↑ LogExpFunctions v0.3.10 ⇒ v0.3.12
  [931e9471] + MechanismGeometries v0.7.1
  [6ad125db] + MeshCatMechanisms v0.8.2
  [69de0a69] ↑ Parsers v2.2.3 ⇒ v2.3.0
  [21216c6a] ↑ Preferences v1.2.5 ⇒ v1.3.0
  [47a9eef4] ↑ SparseDiffTools v1.20.2 ⇒ v1.21.0
  [aedffcd0] ↑ Static v0.5.6 ⇒ v0.6.0
  [a759f4b9] ↑ TimerOutputs v0.5.16 ⇒ v0.5.17
  [2e619515] ↑ Expat_jll v2.4.4+0 ⇒ v2.4.8+0
Precompilin